# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup postings on a variety of topics. You'll train classifiers to distinguish between the topics based on the text of the posts. Whereas with digit classification, the input is relatively dense: a 28x28 matrix of pixels, many of which are non-zero, here we'll represent each document with a "bag-of-words" model. As you'll see, this makes the feature representation quite sparse -- only a few words of the total vocabulary are active in any given document. The bag-of-words assumption here is that the label depends only on the words; their order is not important.

The SK-learn documentation on feature extraction will prove useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [2]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
#from sklearn.grid_search import GridSearchCV

from sklearn.model_selection import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

from sklearn.metrics import f1_score
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
sw = stopwords.words("english")

np.random.seed(50)

[nltk_data] Downloading package punkt to /Users/venky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/venky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Load the data, stripping out metadata so that we learn classifiers that only use textual features. By default, newsgroups data is split into train and test sets. We further split the test so we have a dev set. Note that we specify 4 categories to use for this project. If you remove the categories argument from the fetch function, you'll get all 20 categories.

In [3]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

num_test = len(newsgroups_test.target)
test_data, test_labels = newsgroups_test.data[int(num_test/2):], newsgroups_test.target[int(num_test/2):]
dev_data, dev_labels = newsgroups_test.data[:int(num_test/2)], newsgroups_test.target[:int(num_test/2)]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print('training label shape:', train_labels.shape)
print('test label shape:', test_labels.shape)
print('dev label shape:', dev_labels.shape)
print('labels names:', newsgroups_train.target_names)

training label shape: (2034,)
test label shape: (677,)
dev label shape: (676,)
labels names: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']


(1) For each of the first 5 training examples, print the text of the message along with the label.

In [4]:
def P1(num_examples=5):
    for i in range (num_examples):
        
        print ("This is an example of Group:", train_labels[i], \
               "type: ",newsgroups_train.target_names[train_labels[i]])
        print ("_" * 80)
        print (train_data[i])
        print ("_" * 80)
        print ("\n")
        
        #print (train_labels[i])
P1()

This is an example of Group: 1 type:  comp.graphics
________________________________________________________________________________
Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
________________________________________________________________________________


This is an example of Group: 3 type:  talk.religion.misc
________________________________________________________________________________


Seems to be, barring evidence to th

(2) Use CountVectorizer to turn the raw training text into feature vectors. You should use the fit_transform function, which makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").

The vectorizer has a lot of options. To get familiar with some of them, write code to answer these questions:

a. The output of the transform (also of fit_transform) is a sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html. What is the size of the vocabulary? What is the average number of non-zero features per example? What fraction of the entries in the matrix are non-zero? Hint: use "nnz" and "shape" attributes.

b. What are the 0th and last feature strings (in alphabetical order)? Hint: use the vectorizer's get_feature_names function.

c. Specify your own vocabulary with 4 words: ["atheism", "graphics", "space", "religion"]. Confirm the training vectors are appropriately shaped. Now what's the average number of non-zero features per example?

d. Instead of extracting unigram word features, use "analyzer" and "ngram_range" to extract bigram and trigram character features. What size vocabulary does this yield?

e. Use the "min_df" argument to prune words that appear in fewer than 10 documents. What size vocabulary does this yield?

f. Using the standard CountVectorizer, what fraction of the words in the dev data are missing from the vocabulary? Hint: build a vocabulary for both train and dev and look at the size of the difference.

In [5]:
def P2():
    vectorizer = CountVectorizer()
    feature_vec = vectorizer.fit_transform(train_data)
    
    print ("\n")
    #print (np.shape(feature_vec[1]))
    
    #Question a. 
    #Size of vocabulary
    print ("-"*80)
    print ("Question 2.a")
    print ("-"*80)
    
    print ("Shape of feature vector: ", np.shape(feature_vec))
    print ("Size of vocabulary: ", np.shape(feature_vec)[1])
    
    #Average size of non-zero feature values
    print ("Number of non-zero features in matrix: ", feature_vec.nnz)
    print ("Size of feature vector matrix: ", np.prod(np.shape(feature_vec)))
    print ("Fraction of non-zero entries in matrix: {0:0.5f}".format(feature_vec.nnz/(np.prod(np.shape(feature_vec)))))
    print ("Average size of non-zero feature values per document: {0:0.2f}"
           .format(feature_vec.nnz/(np.shape(feature_vec))[0]))
    print ("-"*80)
    print ("\n")
    
    #Question b
    print ("-"*80)
    print ("Question 2.b")
    print ("-"*80)
    
    #First and last feature names
    print ("First feature name:", vectorizer.get_feature_names()[0])
    print ("Last feature name:", vectorizer.get_feature_names()[len(vectorizer.get_feature_names()) - 1])
    print ("\n")
    
    #Question c
    print ("-"*80)
    print ("Question 2.c")
    print ("-"*80)
    
    #Specifying own vocabulary with [atheism", "graphics", "space", "religion"]
    my_vocab = ["atheism", "graphics", "space", "religion"]
    my_vectorizer = CountVectorizer(vocabulary=my_vocab)
    my_feature_vec = my_vectorizer.fit_transform(train_data)
    
    print ("Shape of my feature vector: ", np.shape(my_feature_vec))
    print ("Size of my vocabulary: ", np.shape(my_feature_vec)[1])
    
    #Average size of non-zero feature values
    print ("Number of non-zero features in matrix with my vocab: ", my_feature_vec.nnz)
    print ("Size of feature vector matrix with my vocab: ", np.prod(np.shape(my_feature_vec)))
    print ("Fraction of non-zero entries in matrix: {0:0.5f} with my vocab".format(my_feature_vec.nnz/(np.prod(np.shape(my_feature_vec)))))
    print ("Average size of non-zero feature values per document: {0:0.2f} with my vocab"
           .format(my_feature_vec.nnz/(np.shape(my_feature_vec))[0]))
    print ("-"*80)
    print ("\n")
    
    #Question d
    print ("-"*80)
    print ("Question 2.d")
    print ("-"*80)
    
    #Bi-gram vectorizer
    bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),
                                        token_pattern=r'\b\w+\b', 
                                        min_df=1)
    bigram_feature_vec = bigram_vectorizer.fit_transform(train_data)
    print ("Size of bi-gram vocabulary: ", np.shape(bigram_feature_vec)[1])
    
    #Trigram vectorizer
    trigram_vectorizer = CountVectorizer(ngram_range=(1, 3),
                                        token_pattern=r'\b\w+\b', 
                                        min_df=1)
    trigram_feature_vec = trigram_vectorizer.fit_transform(train_data)
    print ("Size of tri-gram vocabulary: ", np.shape(trigram_feature_vec)[1])
    print ("\n")
    
    #Question e
    print ("-"*80)
    print ("Question 2.e")
    print ("-"*80)
    
    #Using mind_df of 10, build unigram vectorizer
    uni_min_df10_vectorizer = CountVectorizer(min_df = 10)
    uni_min_df10_feature_vec = uni_min_df10_vectorizer.fit_transform(train_data)
    print ("Size of unigram vocabulary with df = 10: ", np.shape(uni_min_df10_feature_vec)[1])
    print ("\n")
    
    #Question f
    print ("-"*80)
    print ("Question 2.f")
    print ("-"*80)
    
    #Get vocab for train data using original vectorizer one more time
    train_data_vectorizer = CountVectorizer()
    train_feature_vec = train_data_vectorizer.fit_transform(train_data)
    print ("Size of train data vocabulary: ", np.shape(train_feature_vec)[1])
    
    #Get vocab for dev data using original vectorizer
    dev_data_vectorizer = CountVectorizer()
    dev_feature_vec = dev_data_vectorizer.fit_transform(dev_data)
    print ("Size of dev data vocabulary: ", np.shape(dev_feature_vec)[1])
    
    #find out which keys are in dev data, but not in the training data
    #missing_key_cnt = 0
    #for key in dev_feature_vec : 
    #   if key not in train_feature_vec :
    #       missing_key_cnt += 1
   
    missing_keys = set(dev_data_vectorizer.vocabulary_.keys()) - set(train_data_vectorizer.vocabulary_.keys())
    missing_key_cnt = len(missing_keys)
    print ("Number of missing keys", missing_key_cnt)
            
    print ("Fraction of words from dev data missing in train data vocabulary: {0:.2f}"
           .format(missing_key_cnt/np.shape(dev_feature_vec)[1]))
    print ("\n")

    
    
P2()



--------------------------------------------------------------------------------
Question 2.a
--------------------------------------------------------------------------------
Shape of feature vector:  (2034, 26879)
Size of vocabulary:  26879
Number of non-zero features in matrix:  196700
Size of feature vector matrix:  54671886
Fraction of non-zero entries in matrix: 0.00360
Average size of non-zero feature values per document: 96.71
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
Question 2.b
--------------------------------------------------------------------------------
First feature name: 00
Last feature name: zyxel


--------------------------------------------------------------------------------
Question 2.c
--------------------------------------------------------------------------------
Shape of my feature vector:  (2034, 4)
Size of my vocabulary:  4
Number of no

(3) Use the default CountVectorizer options and report the f1 score (use metrics.f1_score) for a k nearest neighbors classifier; find the optimal value for k. Also fit a Multinomial Naive Bayes model and find the optimal value for alpha. Finally, fit a logistic regression model and find the optimal value for the regularization strength C using l2 regularization. A few questions:

a. Why doesn't nearest neighbors work well for this problem?

b. Any ideas why logistic regression doesn't work as well as Naive Bayes?

c. Logistic regression estimates a weight vector for each class, which you can access with the coef\_ attribute. Output the sum of the squared weight values for each class for each setting of the C parameter. Briefly explain the relationship between the sum and the value of C.

In [6]:
model_lr = LogisticRegression()
model_lr

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [7]:
def P3():
    
    #Unigram CountVectorizer
    vec = CountVectorizer()
    #create training vector
    X_train = vec.fit_transform(train_data)
    Y_train = train_labels
    
    #print shapes
    print ("Shape of training data", np.shape(X_train))
    print ("Shape of training labels", np.shape(Y_train))
    
    #Now let's try out different algorithms with this training set
    #kNN - find optimal value of k using GridSearch
    k_list = [2*k+3 for k in range (25)]
    k_values = {'n_neighbors' : k_list}
    model_knn = KNeighborsClassifier()
    clf_knn = GridSearchCV(model_knn, k_values, cv = 5)
    clf_knn.fit(X_train, Y_train)
    print ("-"*80)
    print ("Trying out Grid Search on k values for KNN Algorithm")
    print ("")
    print(clf_knn.cv_results_['mean_test_score'])
    print("The best k value for KNN:", clf_knn.best_params_)
    print("The best f1 score for KNN", clf_knn.best_score_)
    print ("-"*80)
    print ("\n")

    #Now Multinomial NB
    model_nb = MultinomialNB()
    alpha_values = {'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0, 10]}
    clf_nb = GridSearchCV(model_nb, alpha_values, cv =5)
    clf_nb.fit(X_train, Y_train)
    print ("-"*80)
    print ("Trying out Grid Search on alpha values for Multinomial Naive Bayes Algorithm")
    print(clf_nb.cv_results_['mean_test_score'])
    print("The best alpha value for Multinomial NB", clf_nb.best_params_)
    print("The best f1 score for Multinomial NB", clf_nb.best_score_)
    print ("-"*80)
    print ("\n")
    
    #Logistic Regression
    model_lr = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=500, tol=0.01)
    C_values = {'C': [0.001, 0.01, 0.1, 1, 10, 100,1000]}
    clf_lr = GridSearchCV(model_lr, C_values, cv =5)
    clf_lr.fit(X_train, Y_train)
    print ("-"*80)
    print ("Trying out Grid Search on C values for Logistic Regression")
    print(clf_lr.cv_results_['mean_test_score'])
    print("The best alpha value for Logistic Regression", clf_lr.best_params_)
    print("The best f1 score for Logistic Regression", clf_lr.best_score_)
    print ("-"*80)
    print ("\n")    
    
P3()

Shape of training data (2034, 26879)
Shape of training labels (2034,)
--------------------------------------------------------------------------------
Trying out Grid Search on k values for KNN Algorithm

[0.40363815 0.41494592 0.43608653 0.42822026 0.4346116  0.43952802
 0.43215339 0.42772861 0.44346116 0.44001967 0.42969518 0.43854474
 0.43510324 0.43313668 0.43608653 0.4252704  0.42428712 0.42084562
 0.42625369 0.42281219 0.43117011 0.42576205 0.42772861 0.43166175
 0.43756146]
The best k value for KNN: {'n_neighbors': 19}
The best f1 score for KNN 0.44346116027531957
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
Trying out Grid Search on alpha values for Multinomial Naive Bayes Algorithm
[0.82497542 0.8259587  0.82546706 0.82841691 0.82055064 0.79990167
 0.71435595]
The best alpha value for Multinomial NB {'alpha': 0.01}
The best f1 score for Multinomial NB 0.828416

In [8]:
def P3c():
    
    #Unigram CountVectorizer
    vec = CountVectorizer()
    #create training vector
    X_train = vec.fit_transform(train_data)
    Y_train = train_labels
    #part 3.c
    model_num = 0
    C_values = [0.001, 0.01, 0.1, 1, 10, 100]
    #C_values = [0.001]
    for C_val in C_values:
        model_lr = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=500, C = C_val)
        model_lr.fit(X_train, Y_train)
        print ('C:', C_val, 'Sum of squares:', np.sum(np.square(model_lr.coef_), axis=1))
        
P3c()

C: 0.001 Sum of squares: [0.09426742 0.1574485  0.12489113 0.08813422]
C: 0.01 Sum of squares: [1.52965542 2.02979702 1.86576651 1.36936079]
C: 0.1 Sum of squares: [14.0875465  14.44474257 15.47251658 12.85538743]
C: 1 Sum of squares: [72.15025883 63.39471997 72.84092148 66.80543594]
C: 10 Sum of squares: [212.97983181 179.79229424 205.22497591 200.8300326 ]
C: 100 Sum of squares: [446.48908511 378.31280497 424.27648127 429.89961645]


ANSWER:

(4) Train a logistic regression model. Find the 5 features with the largest weights for each label -- 20 features in total. Create a table with 20 rows and 4 columns that shows the weight for each of these features for each of the labels. Create the table again with bigram features. Any surprising features in this table?

In [9]:
def better_preprocessor(s):
    
    #use stemmer
    ps = PorterStemmer()
    s = ps.stem(s)
    
    
    #Let's first remove digits only since they don't make sense
    #s = re.sub('\d+',"",s)
    s = re.sub('\d+',"digittoken",s)
    
    #remove 2 or more underscores
    s = re.sub('__+', " ", s)
    
    #remove underscores that are at the beginning of a word
    s = re.sub('_',' ',s)
    
    #remote words less then 2 letters
    #s = re.sub('\s+\w\w\s+'," ",s)
    #s = re.sub('^\w\w\s+'," ",s)
    
    #remove greetings
    s = re.sub('\s+hi\s+',"",s,flags= re.IGNORECASE)
    s = re.sub('\s+thanks\s+',"",s,flags= re.IGNORECASE)
    s = re.sub('\s+cheers\s+',"",s,flags= re.IGNORECASE)
    s = re.sub('^hi\s+',"",s,flags= re.IGNORECASE)
    s = re.sub('^thanks\s+',"",s,flags= re.IGNORECASE)
    s = re.sub('^cheers\s+',"",s,flags= re.IGNORECASE)
    
    #replace n't with not
    s = re.sub('n\'t',' not',s)
    
    #remove names??
    s = re.sub('bobby','',s, flags = re.IGNORECASE)
    s = re.sub('kent','',s, flags = re.IGNORECASE)
    
    return s

def P4(grams = 1, preproc = 1):
    
    if (grams == 1) : 
        #Unigram CountVectorizer
        print ("Using Unigram vectorizer")
        vec = CountVectorizer()
        #stop_words='english')
    else :
        #Bigram
        print ("Using Bigram vectorizer")
        vec = CountVectorizer(ngram_range=(2, 2), token_pattern=r'\b\w+\b', min_df=1)
                              #stop_words='english')
        
    #create training vector
    if (preproc == 1) : 
        print ("Turning on pre-processor")
        train_data_preproc = []
        for i in range (len(train_data)) :
            train_data_preproc.append(better_preprocessor(train_data[i]))
        X_train = vec.fit_transform(train_data_preproc)
    else:
        X_train = vec.fit_transform(train_data)
        
    Y_train = train_labels
    
    #Logistic Regression with C = 0.1
    model_lr = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=500, C = 0.1)
    model_lr.fit(X_train, Y_train)
    
    #Get model_lr.coef_ and look at the top 5 values for each row
    for i in range (np.shape(model_lr.coef_)[0]):
        print (newsgroups_train.target_names[i])
        test = model_lr.coef_[i,:]
        
        #print (test[0:4])
        temp = np.argpartition(-test, 5)
        result_args = temp[:5]

        temp = np.partition(-test, 5)
        result = -temp[:5]
    
        print (result_args)
        print (result)
        for i in result_args.tolist() :
            print (vec.get_feature_names()[i])
    
P4(grams = 1)
P4(grams = 2)

Using Unigram vectorizer
Turning on pre-processor
alt.atheism
[17994  1491  1495  9877 11320]
[0.42824964 0.40434063 0.35918161 0.28948202 0.28850476]
religion
atheism
atheists
him
islam
comp.graphics
[ 8067  9221 10426  5859  8070]
[0.49648433 0.76211146 0.5095598  0.45843182 0.39522489]
file
graphics
image
digittokend
files
sci.space
[12172 15211 14308 20120 20123]
[0.35985263 0.44941994 0.44134161 1.00745951 0.31451187]
launch
orbit
nasa
space
spacecraft
talk.religion.misc
[ 9070  3514  3511  2363 14894]
[0.41477418 0.41975703 0.40791805 0.33318227 0.33040952]
god
christians
christian
blood
objective
Using Bigram vectorizer
Turning on pre-processor
alt.atheism
[ 82525  47701  78014  15266 167224]
[0.31647999 0.2815832  0.2303864  0.22505229 0.2184146 ]
is not
do not
in this
are you
to say
comp.graphics
[ 76906  74752 157039  92432 116724]
[0.43386168 0.34354633 0.33224502 0.46383247 0.33204205]
in advance
i need
the digittoken
looking for
out there
sci.space
[159973 158626 138593  1

ANSWER:

(5) Try to improve the logistic regression classifier by passing a custom preprocessor to CountVectorizer. The preprocessing function runs on the raw text, before it is split into words by the tokenizer. Your preprocessor should try to normalize the input in various ways to improve generalization. For example, try lowercasing everything, replacing sequences of numbers with a single token, removing various other non-letter characters, and shortening long words. If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular. With your new preprocessor, how much did you reduce the size of the dictionary?

For reference, I was able to improve dev F1 by 2 points.

In [159]:
#def empty_preprocessor(s):
#    return s

def better_preprocessor(s):
    
    #use stemmer
    #ps = PorterStemmer()
    #s = ps.stem(s)

    #remove commonly used english words
    
    s = re.sub('\bthe\b', '', s, flags=re.IGNORECASE)
    s = re.sub('\band\b', '', s, flags=re.IGNORECASE)
    s = re.sub('\bit\b', '', s, flags=re.IGNORECASE)
    s = re.sub('\bi\b','',s, flags=re.IGNORECASE)
    s = re.sub('\bis\b','',s, flags=re.IGNORECASE)
    s = re.sub('\bnot\b','',s, flags=re.IGNORECASE)
    s = re.sub('\bwas\b','',s, flags=re.IGNORECASE)
    s = re.sub('\bto\b','',s, flags=re.IGNORECASE)
    
    #Let's first remove digits only since they don't make sense
    s = re.sub('\d+',"digittoken",s)
    
    #remove 2 or more underscores
    s = re.sub('__+', " ", s)
    
    #remove underscores that are at the beginning of a word
    s = re.sub('_',' ',s)
    
    #replace n't with not
    s = re.sub('n\'t',' not',s)
    
    #remove greetings
    s = re.sub('\bhi\b',"",s,flags= re.IGNORECASE)
    s = re.sub('\bthanks\b',"",s,flags= re.IGNORECASE)
    s = re.sub('\bcheers\b',"",s,flags= re.IGNORECASE)
    s = re.sub('^hi\s+',"",s,flags= re.IGNORECASE)
    s = re.sub('^thanks\s+',"",s,flags= re.IGNORECASE)
    s = re.sub('^cheers\s+',"",s,flags= re.IGNORECASE)
    
    #remove names??
    s = re.sub('bobby','',s, flags = re.IGNORECASE)
    s = re.sub('kent','',s, flags = re.IGNORECASE)
    
    #use stemmer
    #ps = PorterStemmer()
    #s = ps.stem(s)
    
    #ignore case
    s = s.lower()
    
    #remove stop words
    stop_word_cnt = 0
    #for word in sw:
        #w = " " + word + " "
        #print (w)
        #s = s.replace(w, '')
        #s = re.sub(w,'', s, flags = re.IGNORECASE)
        #stop_word_cnt += 1
    
    return s

def P5(preproc = 1, debug = 0):
    
    if (preproc == 1) : 
        train_data_preproc = []
        dev_data_preproc = []
        
        print ("Turning on pre-processor")
        for i in range (len(train_data)) :
            train_data_preproc.append(better_preprocessor(train_data[i]))
        #vectorizer = CountVectorizer(stop_words='english')
        vectorizer = CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b', min_df=1, 
                                     stop_words='english',
                                    preprocessor = better_preprocessor)
        #X_train = vectorizer.fit_transform(train_data_preproc)
        X_train = vectorizer.fit_transform(train_data)
        print("Number of features:", len(vectorizer.get_feature_names()))
        
        for i in range (len(dev_data)) :
            dev_data_preproc.append(better_preprocessor(dev_data[i]))
            
        X_dev = vectorizer.transform(dev_data_preproc)
    else:
        print ("No preprocessing")
        #Let's look at the baseline vectorizer and see what can be improved
        #vectorizer = CountVectorizer(stop_words='english')
        vectorizer = CountVectorizer(ngram_range=(1, 1), token_pattern=r'\b\w+\b', min_df=1)
        #stop_words='english')
        X_train = vectorizer.fit_transform(train_data)
        X_dev = vectorizer.transform(dev_data)
        print("Number of features:", len(vectorizer.get_feature_names()))
    
    #First 200 feature names
    if (debug == 1) :
        for i in range (50):
            print ("i:", i, "Name:", vectorizer.get_feature_names()[i+4000])
    else: 
    
        Y_train = train_labels
        Y_dev = dev_labels
    
        #Logistic Regression
        #model_lr = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=500)
        #C_values = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
        #C_values = {'C': [0.1, 1, 10]}
        #clf_lr = GridSearchCV(model_lr, C_values, cv =5)
        #clf_lr.fit(X_train, Y_train)
        #print ("-"*80)
        #print ("Trying out Grid Search on C values for Logistic Regression")
        #print(clf_lr.cv_results_['mean_test_score'])
        #print("The best alpha value for Logistic Regression", clf_lr.best_params_)
        #print("The best f1 score for Logistic Regression", clf_lr.best_score_)
        
        #print ("-"*80)
        #print ("\n") 
        
        
        #Use same vectorizer to transform dev_data
        
        C_val = [0.01, 0.1, 0.5, 1, 2, 5, 10]
    
        for C in C_val :
            #Logistic Regression with C = 100
            model_lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=500, C = C, tol = 0.0001)
            model_lr.fit(X_train, Y_train)
    
            predicted_labels = model_lr.predict(X_dev)
            print ("C Value:", C, "Number of mispredictions:", np.sum(predicted_labels != Y_dev))
            #print (predicted_labels[0:30])
            #print (Y_dev[0:30])
            print ("C Value:", C, "F1 score: ", f1_score(Y_dev, predicted_labels, average = 'weighted'))
    
P5(preproc = 0, debug = 0)
P5(preproc = 1, debug = 0)

No preprocessing
Number of features: 26919
C Value: 0.01 Number of mispredictions: 223
C Value: 0.01 F1 score:  0.6508456650730315
C Value: 0.1 Number of mispredictions: 209
C Value: 0.1 F1 score:  0.6829596797981359
C Value: 0.5 Number of mispredictions: 211
C Value: 0.5 F1 score:  0.683336695710661
C Value: 1 Number of mispredictions: 215
C Value: 1 F1 score:  0.6778823764330951
C Value: 2 Number of mispredictions: 210
C Value: 2 F1 score:  0.6854379150024141
C Value: 5 Number of mispredictions: 208
C Value: 5 F1 score:  0.6888428587454576
C Value: 10 Number of mispredictions: 208
C Value: 10 F1 score:  0.6888205791542036
Turning on pre-processor
Number of features: 24049
C Value: 0.01 Number of mispredictions: 204
C Value: 0.01 F1 score:  0.6763588607962638
C Value: 0.1 Number of mispredictions: 185
C Value: 0.1 F1 score:  0.7186049520053149
C Value: 0.5 Number of mispredictions: 190
C Value: 0.5 F1 score:  0.7132986931149863
C Value: 1 Number of mispredictions: 194
C Value: 1 F1 sc

(6) The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. That is, logistic regression seeks the set of weights that minimizes errors in the training data AND has a small size. The default regularization, L2, computes this size as the sum of the squared weights (see P3, above). L1 regularization computes this size as the sum of the absolute values of the weights. The result is that whereas L2 regularization makes all the weights relatively small, L1 regularization drives lots of the weights to 0, effectively removing unimportant features.

Train a logistic regression model using a "l1" penalty. Output the number of learned weights that are not equal to zero. How does this compare to the number of non-zero weights you get with "l2"? Now, reduce the size of the vocabulary by keeping only those features that have at least one non-zero weight and retrain a model using "l2".

Make a plot showing accuracy of the re-trained model vs. the vocabulary size you get when pruning unused features by adjusting the C parameter.

Note: The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.01 (the default is .0001).

In [114]:
def P6():
    
    # Initialize seed
    np.random.seed(50)
    
    #vec = CountVectorizer()
    vec = TfidfVectorizer(stop_words = 'english')
    X_train = vec.fit_transform(train_data)
    Y_train = train_labels
    
    X_dev = vec.transform(dev_data)
    Y_dev = dev_labels
    
    C_val = [0.01, 0.1, 1, 10, 100, 1000]
    
    #C_val = [0.25, 0.5, 0.75, 1, 1.25, 1.5]
    
    #C_val = [0.75]
    
    #First solve for penalty = l2
    print ("Penalty: l2")
    print ("-" * 80)
    for C in C_val :
        model_lr_l2 = LogisticRegression(solver = 'lbfgs', 
                                        multi_class='auto', 
                                        max_iter=500, 
                                        C = C, 
                                        tol = 0.01,
                                        penalty = 'l2'
                                        )
        
        #Fit the model with training data
        model_lr_l2.fit(X_train, Y_train)
        
        pred_l2_dev = model_lr_l2.predict(X_dev)
        model_l2_score = f1_score(Y_dev, pred_l2_dev, average = 'weighted')
        
        zero_coef_sum_l2 = np.sum(model_lr_l2.coef_ < 0.0000000001)
        
        print ("C value: ", C, "F1 Score: ", model_l2_score, 
                "Number of zero coefficients:", zero_coef_sum_l2)
        
    print ("-" * 80)
    
    #Solve for penalty = l1
    print ("Penalty: l1")
    print ("-" * 80)
    for C in C_val :
        model_lr_l1 = LogisticRegression(solver = 'saga',
                                      multi_class='auto', 
                                      max_iter=500, 
                                      C = C, 
                                      tol = 0.01,
                                      penalty = 'l1'
                                      )
            
        #Fit the model with training data
        model_lr_l1.fit(X_train, Y_train)
        
        pred_l1_dev = model_lr_l1.predict(X_dev)
        model_l1_score = f1_score(Y_dev, pred_l1_dev, average = 'weighted')
        
        zero_coef_sum_l1 = np.sum(model_lr_l1.coef_ < 0.0000000001)
        
        print ("C value: ", C, "F1 Score: ", model_l1_score, 
                "Number of zero coefficients:", zero_coef_sum_l1)
        
    print ("-" * 80)   
            
    #Choose a particular C value for analysis based on above results
    #Choosing 0.01 since it has the least number of elements
    model_lr_l1_opt = LogisticRegression(solver = 'saga',
                                      multi_class='auto', 
                                      max_iter=500, 
                                      C = 1000, #1, #1, #1, #0.01
                                      tol = 0.01,
                                      penalty = 'l1'
                                     )
    
    #Fit the model with training data
    model_lr_l1_opt.fit(X_train, Y_train)
    
    #Use only the features that have at least one non-zero coefficient value
    non_zero_coef_sum = model_lr_l1_opt.coef_ > 0.0000000001
    print ("Shape of non_zero_coef_sum", np.shape(non_zero_coef_sum))
    
    non_zero_feature_vec = np.any (non_zero_coef_sum, axis = 0)
    
    #print (non_zero_coef_sum[1:5, 1:10])
    #print (non_zero_feature_vec[1:5])
    
    print ("Shape of non_zero_feature_vec", np.shape(non_zero_feature_vec))
    print ("Number of non zero features", np.sum(non_zero_feature_vec))
    
    l1_opt_vocab = []
    for i in range(len(vec.get_feature_names())) :
        if (non_zero_feature_vec[i]) :
            l1_opt_vocab.append(vec.get_feature_names()[i])
            #if (penalty == 'l1') :
            #    print (vec.get_feature_names()[i])
            
    #Now use only these features to compute with l2
    #opt_vec = CountVectorizer(vocabulary=l1_opt_vocab)
    
    train_data_preproc = []
    for i in range (len(train_data)) :
            train_data_preproc.append(better_preprocessor(train_data[i]))
    dev_data_preproc = []
    for i in range (len(dev_data)) :
            dev_data_preproc.append(better_preprocessor(dev_data[i]))
            
    #tfidf_vec = TfidfVectorizer(stop_words = 'english')
    
    opt_vec = TfidfVectorizer(vocabulary=l1_opt_vocab, stop_words='english')
    X_train_opt = opt_vec.fit_transform(train_data_preproc)
    X_dev_opt = opt_vec.transform(dev_data_preproc)
    
    C_val = [0.01, 0.1, 1, 10, 100, 1000]
    for C in C_val:
        model_lr_opt = LogisticRegression(solver = 'lbfgs', 
                                          multi_class='auto', 
                                          max_iter=500, 
                                          C = C, 
                                          tol = 0.01,
                                          penalty = 'l2'
                                        )
        
        model_lr_opt.fit(X_train_opt, Y_train)
    
        pred_dev_opt = model_lr_opt.predict(X_dev_opt)
        model_score_opt = f1_score(Y_dev, pred_dev_opt, average = 'weighted')
        
        print ("C value: ", C, "F1 Score: ", model_score_opt)
    
#P6(penalty = 'l2')
P6()

Penalty: l2
--------------------------------------------------------------------------------
C value:  0.01 F1 Score:  0.39687014102662566 Number of zero coefficients: 74850


/Users/venky/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


C value:  0.1 F1 Score:  0.5943310825587658 Number of zero coefficients: 75062
C value:  1 F1 Score:  0.7319417804875549 Number of zero coefficients: 75584
C value:  10 F1 Score:  0.7745841499146137 Number of zero coefficients: 75797
C value:  100 F1 Score:  0.7725091825398225 Number of zero coefficients: 75872
C value:  1000 F1 Score:  0.7717379636396263 Number of zero coefficients: 75744
--------------------------------------------------------------------------------
Penalty: l1
--------------------------------------------------------------------------------
C value:  0.01 F1 Score:  0.13390025359256127 Number of zero coefficients: 106304
C value:  0.1 F1 Score:  0.2516600544546179 Number of zero coefficients: 106302


/Users/venky/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


C value:  1 F1 Score:  0.6940259935581442 Number of zero coefficients: 106097
C value:  10 F1 Score:  0.7295474975631954 Number of zero coefficients: 103476
C value:  100 F1 Score:  0.7530462413996881 Number of zero coefficients: 92256
C value:  1000 F1 Score:  0.765174353744384 Number of zero coefficients: 81075
--------------------------------------------------------------------------------
Shape of non_zero_coef_sum (4, 26576)
Shape of non_zero_feature_vec (26576,)
Number of non zero features 20703


/Users/venky/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


C value:  0.01 F1 Score:  0.3965172282974863
C value:  0.1 F1 Score:  0.5852661918095813
C value:  1 F1 Score:  0.7280633759703681
C value:  10 F1 Score:  0.7694598854823191
C value:  100 F1 Score:  0.7751379625081135
C value:  1000 F1 Score:  0.7787621059998863


(7) Use the TfidfVectorizer -- how is this different from the CountVectorizer? Train a logistic regression model with C=100.

Make predictions on the dev data and show the top 3 documents where the ratio R is largest, where R is:

maximum predicted probability / predicted probability of the correct label

What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

In [156]:
def P7():
    
    train_data_preproc = []
    for i in range (len(train_data)) :
            train_data_preproc.append(better_preprocessor(train_data[i]))
    dev_data_preproc = []
    for i in range (len(dev_data)) :
            dev_data_preproc.append(better_preprocessor(dev_data[i]))
            
    tfidf_vec = TfidfVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1, stop_words = 'english')
    
    #create training vector
    X_train = tfidf_vec.fit_transform(train_data_preproc)
    Y_train = train_labels
    
    #Use same vectorizer to transform dev_data
    X_dev = tfidf_vec.transform(dev_data_preproc)
    Y_dev = dev_labels
    
    #C_val = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
    C_val = [0.1]
    
    for C in C_val :
        #Logistic Regression with C = 100
        model_lr = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=500, C = C, tol = 0.01)
        model_lr.fit(X_train, Y_train)
    
        predicted_labels = model_lr.predict(X_dev)
        
        print ("C Value:", C, "F1 score: ", f1_score(Y_dev, predicted_labels, average = 'weighted'))
        
    #Using C = 1000 to find out the ratio where predictions were correct but ratios we lowest
    model_lr = LogisticRegression(solver='lbfgs', multi_class='auto', max_iter=500, C = 1000, tol = 0.01)
    model_lr.fit(X_train, Y_train)
    
    predicted_labels = model_lr.predict(X_dev)
    predicted_prob = model_lr.predict_proba(X_dev)
    
    
    #Look at the first 10 examples
    print (predicted_prob[0:10], predicted_labels[0:10])
    
    #max_predicted_prob = np.max(predicted_prob)
    #print ("Max predicted prob for C=1000", max_predicted_prob)
    
    #Find whether the predictions were correct
    #corr_preds = predicted_labels == dev_labels
    
    #print ("Number of correct predictions", sum(corr_preds))
    
    #Use only those rows that have correct_preds == 1
    
    #corr_pred_probs = predicted_prob[corr_preds]
    #dev_data_corr_pred = dev_data[corr_preds]
    
    #corr_pred_labels = predicted_labels[corr_preds]
    
    #Get the max_probability across the 4 categories
    #corr_pred_probs_max = np.max(corr_pred_probs, axis = 1)
    
    #
    #print ("Number of correctly predicted labels", np.shape(corr_pred_probs_max))
    
    #print ("Smallest probabilities for correct labels:", (np.sort(corr_pred_probs_max))[0:3])
    #print ("The indices with smallest probability", (np.argsort((corr_pred_probs_max))[0:3]))
    
    #top3_ratio_elems = np.argsort((corr_pred_probs_max))[0:3]
    #print ("The labels with the smallest probability", corr_pred_labels[top3_ratio_elems])
    
    #print ("Top3 documents with max ratio of max probability to probability of correct label")
    #for i in range (3):
    #    print ("Document:", i)
    #    print ("-"*80)
     #   #print ("Index:", top3_ratio_elems.tolist()[i] )
     #   #print (X_dev_corr_pred[top3_ratio_elems[i]])
    #    print (dev_data_corr_pred[i])
     #   print ("-"*80)
    
    
    
P7()

C Value: 0.1 F1 score:  0.5712580763764685
[[8.89275884e-03 1.28235503e-02 9.70768343e-01 7.51534783e-03]
 [2.12370669e-03 9.90809822e-01 5.68601154e-03 1.38045970e-03]
 [3.70379884e-05 9.98230315e-01 1.61426647e-03 1.18380093e-04]
 [2.29049342e-04 9.95083581e-01 3.69516903e-03 9.92200659e-04]
 [1.06031752e-03 9.97016451e-01 1.48586527e-03 4.37366622e-04]
 [3.91884454e-04 9.91390895e-01 7.88109934e-03 3.36121526e-04]
 [1.19876249e-01 1.42058755e-01 6.17025106e-01 1.21039891e-01]
 [1.70160958e-04 1.45868753e-04 9.99482656e-01 2.01313809e-04]
 [9.91587566e-01 2.25654647e-03 4.46697459e-03 1.68891275e-03]
 [2.57401324e-01 5.49528930e-02 4.69366383e-01 2.18279400e-01]] [2 1 1 1 1 1 2 2 0 2]


ANSWER:

(8) EXTRA CREDIT

Try implementing one of your ideas based on your error analysis. Use logistic regression as your underlying model.

In [120]:
X = np.array([[1,2],[3,4]])


In [121]:
1/X

array([[1.        , 0.5       ],
       [0.33333333, 0.25      ]])

In [122]:
23/X

array([[23.        , 11.5       ],
       [ 7.66666667,  5.75      ]])